# Applying a Different Augmentation Strategy Per Class

Let's say we wished to augment the MNIST dataset, but you wished to use a generator to supply a neural network with data. 

Ordinarily you could write a pipeline that would augment all the data, regardless of the class. However with MNIST you might want to have different pipelines for each of the 10 different classes. 

For example, it would make sense to flip images for the figure 8 both horizontally and vertically and still end up with feasible data. The figure 3 could be flipped vertically but not horizontally. Conversely, the figure 4 should not be flipped either horizontally or vertically. 

We can do this by creating 10 different pipelines, and adding or removing the appropriate operations from each pipeline as required.

Augmentor does not support this natively, but it can be performed easily enough, and here we will learn how. 

First we import the required libraries:

In [1]:
import Augmentor
import numpy as np
import os
import glob
import random
import collections

from PIL import Image

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import mnist

random.seed(0)

## 1. Point to a Root Directory

Your root directory must contain subdirectories, one for each class in your machine learning classification problem:

In [2]:
root_directory = 'data/Train/*'

# root_directory = "/home/marcus/Documents/mnist/train/*"

## 2. Scan for folders in the root directory

We use `glob.glob()` to scan for all files in the `root_directory` and only choose those that are directories. These will be out classes:

In [3]:
folders = []
for f in glob.glob(root_directory):
    if os.path.isdir(f):
        folders.append(os.path.abspath(f))

print("Folders (classes) found: %s " % [os.path.split(x)[1] for x in folders])

Folders (classes) found: ['Scab_Apple', 'Normal_Apple', 'Blotch_Apple', 'Rot_Apple'] 


## 3. Create a pipeline for each class

Now we create a pipeline object for each of the classes. MNIST consists of 10 digits, and each digit represents one class:

In [4]:
pipelines = {}
for folder in folders:
    print("Folder %s:" % (folder))
    pipelines[os.path.split(folder)[1]] = (Augmentor.Pipeline(folder))
    print("\n----------------------------\n")

Folder /Users/macbookpro/MakeAiWork2/projects/apple_disease_classification/data/Train/Scab_Apple:
Initialised with 85 image(s) found.
Output directory set to /Users/macbookpro/MakeAiWork2/projects/apple_disease_classification/data/Train/Scab_Apple/output.
----------------------------

Folder /Users/macbookpro/MakeAiWork2/projects/apple_disease_classification/data/Train/Normal_Apple:
Initialised with 67 image(s) found.
Output directory set to /Users/macbookpro/MakeAiWork2/projects/apple_disease_classification/data/Train/Normal_Apple/output.
----------------------------

Folder /Users/macbookpro/MakeAiWork2/projects/apple_disease_classification/data/Train/Blotch_Apple:
Initialised with 116 image(s) found.
Output directory set to /Users/macbookpro/MakeAiWork2/projects/apple_disease_classification/data/Train/Blotch_Apple/output.
----------------------------

Folder /Users/macbookpro/MakeAiWork2/projects/apple_disease_classification/data/Train/Rot_Apple:
Initialised with 114 image(s) found.

We can summarise what was scanned:

In [5]:
for p in pipelines.values():
    print("Class %s has %s samples." % (p.augmentor_images[0].class_label, len(p.augmentor_images)))

Class Scab_Apple has 85 samples.
Class Normal_Apple has 67 samples.
Class Blotch_Apple has 116 samples.
Class Rot_Apple has 114 samples.


## 4. Add operations to the pipelines

Here we will add operations to each of the pipelines. Some operations will be applied to all pipelines, others only to some pipelines.

Here we add a rotate operation to all pipelines (and hence will be applied to all digits):

In [6]:
for pipeline in pipelines.values():
    pipeline.rotate(probability=0.5, max_left_rotation=20, max_right_rotation=20)

Here we add some operations that we only want to apply to certain classes. The figure 8 can be flipped horizontally and vertically:

In [10]:
pipelines["Normal_Apple"].flip_left_right(probability=0.5)

## 5. Define a class label / class integer map

The classes will have string labels associated with them, depending on the name of each class's parent folder. Here you must map the names of each of your classes with the 0-based index (which must correspond to the test data of your dataset).

In the case of MNIST this is quite easy, the samples for the digit 0 were stored in a folder 0 and have the text label 0, and so on:

In [12]:
integer_labels = {'Blotch_Apple': 0, 
                  'Normal_Apple': 1, 
                  'Rot_Apple': 2, 
                  'Scab_Apple': 3}

## 6. Define pipeline containers to store the pipelines and additional information

Later we will need each pipeline's 0-based integer label as well as its categorical label (depending on the type of neural network you define):

In [14]:
PipelineContainer = collections.namedtuple('PipelineContainer', 
                                           'label label_integer label_categorical pipeline generator')

pipeline_containers = []

for label, pipeline in pipelines.items():
    label_categorical = np.zeros(len(pipelines), dtype=int)
    label_categorical[integer_labels[label]] = 1
    pipeline_containers.append(PipelineContainer(label, 
                                                 integer_labels[label], 
                                                 label_categorical, 
                                                 pipeline, 
                                                 pipeline.keras_generator(batch_size=1)))

## 7. Define a generator function

Neural networks in Keras can be supplied with a generator to supply training data. Because we have one generator for each pipeline, we need to create "generator of generators":

In [26]:
def multi_generator(pipeline_containers, batch_size):
    while True:
        X = []
        y = []
        for i in range(batch_size):
            pipeline_container = random.choice(pipeline_containers)
            image, _ = next(pipeline_container.generator)
            # image = image.reshape((256, 256, 3)) # Or (1, 28, 28) for channels_first, see Keras' docs.
            X.append(image)
            y.append(pipeline_container.label_categorical)  # Or label_integer if required by network
        X = np.asarray(X)
        y = np.asarray(y)
        yield X, y

## 8. Create the generator object

Create a generator, `g` to pass data randomly from each pipeline (and hence each class) to a neural network:

In [30]:
batch_size = 32

g = multi_generator(pipeline_containers=pipeline_containers, 
                    batch_size=batch_size)  # Here the batch size can be set to any value

To generate a batch of 128 images and labels, at random from a random pipeline defined above, we can use the `next()` function:

In [31]:
X, y = next(g)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/macbookpro/MakeAiWork2/projects/apple_disease_classification/data/Train/Rot_Apple/Rot_apple32.jpg'

We can confirm that we are receiving images in batches of 128 and that the labels correspond to the images in each pipeline:

In [29]:
print("%s images returned. %s labels returned.") % (np.shape(X)[0], len(y))

%s images returned. %s labels returned.


TypeError: unsupported operand type(s) for %: 'NoneType' and 'tuple'

We can use PIL to view the augmented images and cofirm the labels match (note that PIL requires images to be specified differently to how Keras expects data, hence some preprocessing of the data must be performed):

In [15]:
image_index = 3  # Take image index 3 from the batch

x_array = X[image_index]
x_array = x_array.reshape((28,28))
x_array = x_array * 255
x_array = x_array.astype(np.uint8)
Image.fromarray(x_array)

The label below should correspond to the image output above:

In [16]:
print("Image label: %s" % (np.nonzero(y[image_index])[0][0]))

Image label: 4


## 9. Train a neural network with the generator

Last, we train a neural network with the differing pipelines for each class.

First we define a neural network:

In [17]:
num_classes = len(pipelines)
input_shape = (28, 28, 1)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Once a network has been defined, you can compile it so that the model is ready to be trained with data:

In [18]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

Using the same batch size as the generator above, we can begin to train the neural network:

In [19]:
h = model.fit_generator(g, steps_per_epoch=50000/batch_size, epochs=10, verbose=1)

Epoch 1/10
390/390 [==============================] - 31s - loss: 0.4767 - acc: 0.8502    
Epoch 2/10
390/390 [==============================] - 31s - loss: 0.1364 - acc: 0.9595    
Epoch 3/10
390/390 [==============================] - 30s - loss: 0.1029 - acc: 0.9691    
Epoch 4/10
390/390 [==============================] - 30s - loss: 0.0879 - acc: 0.9740    
Epoch 5/10
390/390 [==============================] - 30s - loss: 0.0778 - acc: 0.9771    
Epoch 6/10
390/390 [==============================] - 30s - loss: 0.0689 - acc: 0.9784    
Epoch 7/10
390/390 [==============================] - 30s - loss: 0.0650 - acc: 0.9804    
Epoch 8/10
390/390 [==============================] - 31s - loss: 0.0632 - acc: 0.9808    
Epoch 9/10
390/390 [==============================] - 30s - loss: 0.0594 - acc: 0.9828    
Epoch 10/10
390/390 [==============================] - 30s - loss: 0.0531 - acc: 0.9845    


## Conclusion

Certain tasks may require different augmentation strategies on a class-by-class basis. The procedure above allows you to do this using Augmentor and Keras.